## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-30-01-15-52 +0000,bbc,Protesters and police clash after death of tax...,https://www.bbc.com/news/articles/cjw6el72xd4o...
1,2025-08-30-01-13-53 +0000,nypost,NJ Little Leaguer in bat flip flap raises near...,https://nypost.com/2025/08/29/sports/nj-little...
2,2025-08-30-01-02-45 +0000,bbc,Barrier being built around besieged Sudan city...,https://www.bbc.com/news/articles/c4gz1ekdee3o...
3,2025-08-30-01-00-00 +0000,wsj,Minnesota Gov. Tim Walz campaigned on the nati...,https://www.wsj.com/politics/elections/we-aske...
4,2025-08-30-01-00-00 +0000,wsj,After months of pushing out career government ...,https://www.wsj.com/politics/policy/trump-admi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
53,trump,63
222,new,22
58,court,17
537,will,13
214,shooting,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
19,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...,137
122,2025-08-29-19-45-00 +0000,wsj,A federal judge signaled that she will move qu...,https://www.wsj.com/politics/policy/judge-weig...,135
55,2025-08-29-22-04-53 +0000,nypost,Trump’s tariffs ruled illegal in stunning blow...,https://nypost.com/2025/08/29/us-news/trumps-t...,132
52,2025-08-29-22-22-47 +0000,cbc,U.S. appeals court finds Trump tariffs unlawfu...,https://www.cbc.ca/news/world/appeals-court-tr...,129
290,2025-08-29-07-56-31 +0000,wsj,"Stock Market News, Aug. 29, 2025: Appeals Cour...",https://www.wsj.com/livecoverage/stock-market-...,123


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
19,137,2025-08-29-23-59-00 +0000,wsj,A federal appeals court struck down the Trump ...,https://www.wsj.com/politics/policy/appeals-co...
122,61,2025-08-29-19-45-00 +0000,wsj,A federal judge signaled that she will move qu...,https://www.wsj.com/politics/policy/judge-weig...
316,57,2025-08-29-03-49-25 +0000,bbc,New video appears to show New Zealand fugitive...,https://www.bbc.com/news/articles/c4g6knz7e65o...
209,47,2025-08-29-14-03-22 +0000,nypost,California Gov. Newsom says crime is higher in...,https://nypost.com/2025/08/29/us-news/californ...
95,44,2025-08-29-20-47-01 +0000,nypost,Rosie O’Donnell falsely rants that transgender...,https://nypost.com/2025/08/29/us-news/rosie-od...
78,42,2025-08-29-21-28-49 +0000,nypost,"15-year-old NYC boy with long rap sheet, ankle...",https://nypost.com/2025/08/29/us-news/15-year-...
275,42,2025-08-29-09-09-26 +0000,cbc,South Korea sees high-profile indictments of f...,https://www.cbc.ca/news/world/south-korea-form...
287,37,2025-08-29-08-20-36 +0000,nypost,"Cracker Barrel co-founder, 93, slams CEO after...",https://nypost.com/2025/08/29/business/cracker...
326,36,2025-08-29-02-16-31 +0000,nypost,RFK Jr’s deputy expected to replace ousted CDC...,https://nypost.com/2025/08/28/us-news/rfk-jrs-...
308,35,2025-08-29-04-27-33 +0000,nypost,JD Vance blasts Jen Psaki for post declaring p...,https://nypost.com/2025/08/29/us-news/jd-vance...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
